In [ ]:
import pandas as pd
import seaborn as sns
from glob import glob
from matplotlib.font_manager import fontManager, FontProperties

path = "/usr/share/fonts/opentype/linux-libertine/LinBiolinum_R.otf"
fontManager.addfont(path)
sns.set_theme(style="ticks", font=FontProperties(fname=path).get_name(), font_scale=1.2)

In [ ]:
data = pd.concat([pd.read_csv(data, names=[
    "Variant", "Regions", "#Client/Region", "Put Ratio", "Throughput", "Latency"
]) for data in glob('saved/cops/*.txt')], ignore_index=True)
data = data.sort_values(["Variant", "Regions", "#Client/Region"])
data["Latency (ms)"] = data["Latency"] * 1000
data

In [ ]:
f = sns.lineplot(
    data=data[(data["Put Ratio"] == 0.01) & (data["Regions"] == "af+ap+eu+sa+us")],
    x="Throughput", y="Latency (ms)", sort=False,
    hue="Variant", 
    style="Variant", markers=['o'], linewidth=2.5,
    hue_order=["Untrusted", "Replicated", "Quorum", "NitroEnclaves"])
f.set(ylim=(0, 110))
# f.figure.savefig("throughput-latency.pdf", bbox_inches='tight')

In [ ]:
region_data = data[data["Put Ratio"] == 0.01].groupby(["Variant", "Regions"]).max("Throughput").reset_index()
def normalized_throughput(row):
    variant = row.at['Variant']
    # TODO
    if variant == "Replicated":
        return 1
    one_row = region_data[(region_data['Variant'] == variant) & (region_data['Regions'] == "af")]
    one = one_row["Throughput"].values[0]
    return row.at['Throughput'] / one
region_data["Normalized Throughput"] = region_data.apply(normalized_throughput, axis=1)
region_data

In [ ]:
sns.barplot(data=region_data,
    x="Regions", y="Normalized Throughput", hue="Variant",
    hue_order=["Untrusted", "Replicated", "Quorum", "NitroEnclaves"])

In [ ]:
ratio_data = data[data["Regions"] == "af+ap+eu+sa+us"].groupby(["Variant", "Put Ratio"]).max("Throughput").reset_index()
ratio_data

In [ ]:
f = sns.lineplot(data=ratio_data,
    x="Put Ratio", y="Throughput", hue="Variant",
    hue_order=["Untrusted", "Replicated", "Quorum", "NitroEnclaves"],
    style="Variant", markers=['o'])
f.set(xscale="log")